In [1]:
from Utility import df_ops, music_ops, vmd_ops, io_ops, cloud_ops
import pandas as pd
import numpy as np
import pymeshio.common

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Don't wrap to multiple pages
pd.set_option('max_colwidth', None)  # No max column width


c:\Users\daryl\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [10]:
VMD_TO_LOAD = 'inputDataVariety/Last Christmas/motion.vmd'
SAVE_FOLDER = 'data/'
SAVE_FILENAME= 'LastChristmas-cloud'
SONG_TO_LOAD = 'data/SuperDuper/song3_sduper [1].wav'
INPUT_DATA_FOLDER = 'inputDataVariety'

settingsSettings = {
  "settings": {
    "index.knn": True
  }
}
settingsBody = {
  "mappings": {
    "properties": {
      "frame": {
        "type": "knn_vector",
        "dimension": 101
      },
      "animation-name": {
        "type": "text"
      },
      "frame-number": {
        "type": "int"
      }
    }
  }
}

In [11]:
#create_index(client, 'frames-index', settingsSettings.update(settingsBody))

In [12]:
df = vmd_ops.convertVMDToDataFrame(vmd_ops.loadVmdFromFile(VMD_TO_LOAD))
df = vmd_ops.filterDataframeForCoreBoneNames(df)

In [13]:
pine = cloud_ops.initPinecone()
cloud_ops.getPineconeIndexStats(pine, 'frames-index')

IndexDescription(name='frames-index', metric='cosine', replicas=1, dimension=101.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')


In [14]:
def getMissingBones(df, boneList):
  df_bone_names_set = set(df['name'])
  boneList_set = set(boneList)
  return list(boneList_set - df_bone_names_set)

def getNextBoneOccurence(df, boneName, startFrame):
  boneDf = df[df['name'] == boneName]
  boneDf = boneDf[boneDf['frame'] >= startFrame]
  if boneDf.empty:
    return None
  else:
    return {"frameNumber": boneDf.iloc[0]['frame'], "position": boneDf.iloc[0]['position'], "rotation": boneDf.iloc[0]['rotation']}
  
def calculateVector3Porportion(currentFrame, lastBoneOccurence, nextBoneOccurence):
  if lastBoneOccurence['frameNumber'] == nextBoneOccurence['frameNumber']:
    return lastBoneOccurence['position']
  else:
    porportion = (currentFrame - lastBoneOccurence['frameNumber']) / (nextBoneOccurence['frameNumber'] - lastBoneOccurence['frameNumber'])
    return pymeshio.common.Vector3(
      lastBoneOccurence['position'].x + (nextBoneOccurence['position'].x - lastBoneOccurence['position'].x) * porportion,
      lastBoneOccurence['position'].y + (nextBoneOccurence['position'].y - lastBoneOccurence['position'].y) * porportion,
      lastBoneOccurence['position'].z + (nextBoneOccurence['position'].z - lastBoneOccurence['position'].z) * porportion
    )

def calculateVector4Porportion(currentFrame, lastBoneOccurence, nextBoneOccurence):
  if lastBoneOccurence['frameNumber'] == nextBoneOccurence['frameNumber']:
    return lastBoneOccurence['rotation']
  else:
    porportion = (currentFrame - lastBoneOccurence['frameNumber']) / (nextBoneOccurence['frameNumber'] - lastBoneOccurence['frameNumber'])
    return pymeshio.common.Vector4(
      lastBoneOccurence['rotation'].x + (nextBoneOccurence['rotation'].x - lastBoneOccurence['rotation'].x) * porportion,
      lastBoneOccurence['rotation'].y + (nextBoneOccurence['rotation'].y - lastBoneOccurence['rotation'].y) * porportion,
      lastBoneOccurence['rotation'].z + (nextBoneOccurence['rotation'].z - lastBoneOccurence['rotation'].z) * porportion,
      lastBoneOccurence['rotation'].w + (nextBoneOccurence['rotation'].w - lastBoneOccurence['rotation'].w) * porportion
    )

import pymeshio.common
def addInterpolationsToEachFrame(df):
  df = df_ops.sortDf(df)
  complement = vmd_ops.getComplement(df)
  boneNames = df['name'].unique().tolist()

  lastBoneList = {boneName: {"frameNumber": None, "position": None, "rotation": None} for boneName in boneNames}
  for boneName in lastBoneList.keys(): #init lastbonelist
    lastBoneList[boneName] = getNextBoneOccurence(df, boneName, 0)
  nextBoneList = {boneName: {"frameNumber": -1, "position": lastBoneList[boneName]['position'], "rotation": lastBoneList[boneName]['rotation']} for boneName in boneNames}

  newFrames = []
  lastFrame = df['frame'].max()
  for currentFrameNumber in range(0, lastFrame + 1):
    frameDf = df[df['frame'] == currentFrameNumber]
    missingBonesOfCurrentFrame = getMissingBones(frameDf, boneNames)
    for missingBone in missingBonesOfCurrentFrame:
      if currentFrameNumber > nextBoneList[missingBone]['frameNumber']:
        lastBoneList[missingBone] = nextBoneList[missingBone]
        nextBoneOccurenceValues = getNextBoneOccurence(df, missingBone, currentFrameNumber)
        nextBoneList[missingBone] = nextBoneOccurenceValues if nextBoneOccurenceValues is not None else nextBoneList[missingBone]
      newFrames.append([
        currentFrameNumber,
        missingBone,
        lastBoneList[missingBone]['position'] if missingBone not in vmd_ops.getBonesWherePositionIsUsed() else calculateVector3Porportion(currentFrameNumber, lastBoneList[missingBone], nextBoneList[missingBone]),
        calculateVector4Porportion(currentFrameNumber, lastBoneList[missingBone], nextBoneList[missingBone])
      ])
      #print(currentFrameNumber, missingBone, nextBoneList[missingBone]['position'], lastBoneList[missingBone]['position'])
  newFramesDf = pd.DataFrame(newFrames, columns=['frame', 'name', 'position', 'rotation'])
  interpolated_df = df.copy()
  interpolated_df = pd.concat([df, newFramesDf])
  interpolated_df = df_ops.sortDf(interpolated_df)
  interpolated_df['complement'] = complement
  return interpolated_df

core_interpolated_df = None
def func(wav, vmd, folderName):
  df = vmd_ops.getDfFromVmdFileName(vmd)
  print(vmd)
  core_df = df[df['name'].isin(vmd_ops.getCoreBoneNames())]
  global core_interpolated_df
  core_interpolated_df = addInterpolationsToEachFrame(core_df)
  #vmd_ops.saveDfToVmdFile(core_interpolated_df, SAVE_FOLDER + '/motionInterpolated.vmd', customComplement=complement)
  folderPath = "\\".join(vmd.split('\\')[:-1])
  print(folderPath)
  df_ops.saveDfToFeather(core_interpolated_df, folderPath  + '\\interpolatedMotion.feather')

io_ops.apply_func_to_inputData(func, "inputDataSingle")

inputDataSingle\PV607 - Secret Police\PV607_HIM_P1_00.vmd
inputDataSingle\PV607 - Secret Police


In [15]:
print(core_interpolated_df.iloc[0]['position'].z)

43.87112808227539


In [16]:

# frames = df[df['name'] == "左ひじ"]
# zero_vector = pymeshio.common.Vector3(0, 0, 0)
# zero_quaternion = pymeshio.common.Vector4(0, 0, 0, 0)

# def is_not_zero_vector(position):
#   return position != zero_quaternion

# filtered_df = df[df['rotation'].apply(is_not_zero_vector)]

# print(filtered_df['name'].unique())
# print(len(vmd_ops.getCoreBoneNames()))
